In [ ]:
# Data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Imputing missing values
from sklearn.impute import KNNImputer

from scipy.stats import chi2_contingency

# Feature engineering
from sklearn.preprocessing import StandardScaler

# Model processing and testing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, plot_roc_curve, precision_score, recall_score

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

# **Table of Contents**
1. [Reading Data](#read)\
     1.1 [Initial Hypotheses](#hypotheses)\
     1.2 [Impute Data](#impute)
2. [EDA](#eda)\
    2.1 [Custom Color Palette](#color)\
    2.2 [Numeric Variables](#numeric)\
    2.3 [Categorical Variables](#categorical)
3. [Bonus EDA- Odds](#bonus)
4. [Feature Engineering](#features)
5. [Model Building](#models)\
    5.1 [Logistic Regression](#logreg)\
    5.2 [Random Forest](#forest)\
    5.3 [K-Nearest Neighbors](#neighbors)\
    5.4 [AdaBoost](#ada)
6. [Conclusion](#conclusion)

## **1. Read Data**

<a id="read"></a>

In [ ]:
df = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
df.head()

<p style="text-align:center;">Determine data shape, count number of null values, find out if there are any negative values.</p>
<p style="text-align:center;">Formulate initial hypothesis; which variables will have the most significant impact on the target variable?</p>

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
s0 = round(df[df['stroke'] == 0].describe(), 2)
s1 = round(df[df['stroke'] == 1].describe(), 2)

pd.concat([s0, s1], axis = 1, keys = ['No Stroke', 'Stroke'])

<p style="text-align:center;">Count null values</p>

In [ ]:
df.isnull().sum()

<p style="text-align:center;">Count negatives</p>

In [ ]:
def count_negatives(data):
    neg_count = 0
    for n in data:
        if type(data) == 'int':
            if n < 0:
               neg_count += 1
    return neg_count

count_negatives(df)

#### 1.1 Initial hypotheses

<a id="hypotheses"></a>

* Age will have a significant effect on stroke regardless of any other variable interaction.
* Previous instances of heart disease will also have a significant effect on stroke.
* Gender will have a positive, albeit weaker, effect.
* Positive instances of smoking status as well as higher bmi will both have a significant effect.

#### 1.2 Impute Data

<a id="impute"></a>

In [ ]:
df_knn = df.copy()
df_knn.head()

In [ ]:
impute = KNNImputer(n_neighbors = 5, weights = 'uniform')
df_knn['bmi'] = impute.fit_transform(df_knn[['bmi']])

<p style="text-align:center;">Check to make sure everything was imputed properly.</p>

In [ ]:
df_knn.isnull().sum()

<p style="text-align:center;">Investigate numeric variables- age, glucose, bmi</p>

<p style="text-align:center;">Histograms for each, their effect on strokes.</p>

<p style="text-align:center;">Potentially graph their effects</p>

<p style="text-align:center;">Investigate the categorical variables- gender, hypertension, heart disease, ever married, work type, smoking status, and stroke</p>

# **2. EDA**

<a id="eda"></a>

#### 2.1 Create Custom Color Palette

<a id="color"></a>

In [ ]:
colors = ["#f1d295", "#c8c14f", "#fa8775", "#ea5f94", "#cd34b5", "#9d02d7"]
palette = sns.color_palette(palette = colors)

sns.palplot(palette, size = 2)
plt.text(-0.5, -0.7, 'Color Palette For This Notebook', size = 20, weight = 'bold')

#### 2.2 Numeric Variables

<a id="numeric"></a>

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
fig.patch.set_facecolor('#faf9f7')
ax.set_facecolor('#faf9f7')

sns.histplot(
    df['age'],
    kde = False,
    color = "#ea5f94"
)

for i in ['top', 'left', 'bottom', 'right']:
    ax.spines[i].set_visible(False)

plt.text(5, 360, r'$\mu$ = '+str(round(df['age'].mean(), 2)), fontsize = 12)
plt.text(5, 343, r'$\sigma$ = '+str(round(df['age'].std(), 2)), fontsize = 12)
plt.title('Frequency of Ages', fontsize = 18, fontweight = 'bold', pad = 10)
plt.xlabel('Age', fontsize = 14, labelpad = 10)
plt.ylabel('Count', fontsize = 14, labelpad = 10)

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
fig.patch.set_facecolor('#faf9f7')
ax.set_facecolor('#faf9f7')

sns.histplot(
    df['avg_glucose_level'],
    color = "#ea5f94",
    kde = False
)

for i in ['top', 'left', 'bottom', 'right']:
    ax.spines[i].set_visible(False)


plt.text(220, 360, r'$\mu$ = '+str(round(df['avg_glucose_level'].mean(), 2)), fontsize = 12)
plt.text(220, 340, r'$\sigma$ = '+str(round(df['avg_glucose_level'].std(), 2)), fontsize = 12)
plt.title('Frequency of Glucose Levels', fontsize = 18, fontweight = 'bold', pad = 10)
plt.xlabel('Average Glucose Level', fontsize = 14, labelpad = 10)
plt.ylabel('Count', fontsize = 14, labelpad = 10)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (12, 7))
fig.patch.set_facecolor('#faf9f7')
ax[0].set_facecolor('#faf9f7')
ax[1].set_facecolor('#faf9f7')

sns.histplot(
    df['bmi'],
    color = "#ea5f94",
    kde = False,
    ax = ax[0]
)

sns.histplot(
    df_knn['bmi'],
    color = "#ea5f94",
    kde = False,
    ax = ax[1]
)

ax[0].text(70, 330, r'$\mu$ = '+str(round(df['bmi'].mean(), 2)), fontsize = 11)
ax[0].text(70, 320, r'$\sigma$ = '+str(round(df['bmi'].std(), 2)), fontsize = 11)
ax[0].set_title('Original BMI Data', fontsize = 16, fontweight = 'bold', pad = 10)
ax[0].set_xlabel('BMI', fontsize = 13)
ax[0].set_ylabel('Count', fontsize = 13)

ax[1].text(70, 500, r'$\mu$ = '+str(round(df_knn['bmi'].mean(), 2)), fontsize = 11)
ax[1].text(70, 485, r'$\sigma$ = '+str(round(df_knn['bmi'].std(), 2)), fontsize = 11)
ax[1].set_title('KNN Imputed BMI Data', fontsize = 16, fontweight = 'bold', pad = 10)
ax[1].set_xlabel('BMI', fontsize = 13)
ax[1].set_ylabel('')

for i in ['top', 'left', 'bottom', 'right']:
    ax[0].spines[i].set_visible(False)
    ax[1].spines[i].set_visible(False)


plt.tight_layout()

In [ ]:
df['bmi'] = df_knn['bmi']
df['bmi'].isnull().sum()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (14,6))
fig.patch.set_facecolor('#faf9f7')

for i in (ax1, ax2, ax3):
    i.set_facecolor('#faf9f7')

sns.kdeplot(
    df['age'][df['stroke'] == 0],
    ax = ax1,
    color = "#c8c14f",
    shade = True,
    alpha = 0.5,
    linewidth = 1.5,
    ec = 'black'
)

sns.kdeplot(
    df['age'][df['stroke'] == 1],
    ax = ax1,
    color = "#cd34b5",
    shade = True,
    alpha = 0.5,
    linewidth = 1.5,
    ec = 'black'
)
ax1.legend(['No Stroke', 'Stroke'], loc = 'upper left')
ax1.set_xlabel('Age', fontsize = 14, labelpad = 10)
ax1.set_ylabel('Density', fontsize = 14, labelpad = 10)

sns.kdeplot(
    df['avg_glucose_level'][df['stroke'] == 0],
    ax = ax2,
    color = "#c8c14f",
    shade = True,
    alpha = 0.5,
    linewidth = 1.5,
    ec = 'black'
)

sns.kdeplot(
    df['avg_glucose_level'][df['stroke'] == 1],
    ax = ax2,
    color = "#cd34b5",
    shade = True,
    alpha = 0.5,
    linewidth = 1.5,
    ec = 'black'
)

ax2.legend(['No Stroke', 'Stroke'])
ax2.set_xlabel('Average Glucose Levels', fontsize = 14, labelpad = 10)
ax2.set_ylabel('')

sns.kdeplot(
    df['bmi'][df['stroke'] == 0],
    ax = ax3,
    color = "#c8c14f",
    shade = True,
    alpha = 0.5,
    linewidth = 1.5,
    ec = 'black'
)

sns.kdeplot(
    df['bmi'][df['stroke'] == 1],
    ax = ax3,
    color = "#cd34b5",
    shade = True,
    alpha = 0.5,
    linewidth = 1.5,
    ec = 'black'
)

ax3.legend(['No Stroke', 'Stroke'])
ax3.set_xlabel('BMI', fontsize = 14, labelpad = 10)
ax3.set_ylabel('')

plt.suptitle('Density of Age, Glucose, and BMI by Stroke', fontsize = 16, fontweight = 'bold')

for i in (ax1, ax2, ax3):
    for j in ['top', 'left', 'bottom', 'right']:
        i.spines[j].set_visible(False)

fig.tight_layout()

<p style="text-align:center;">Scatter plots of numerical variables colored by stroke.</p>

In [ ]:
stroke = df[df['stroke'] == 1]
no_stroke = df[df['stroke'] == 0]

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(16,20))
fig.patch.set_facecolor('#faf9f7')
for j in range(0, 3):
    ax[j].set_facecolor('#faf9f7')

## Age vs Glucose Levels
sns.scatterplot(
    data = no_stroke, x = 'age', y = 'avg_glucose_level', color = '#f1d295',
    alpha = 0.4, ax = ax[0]
)
sns.scatterplot(
    data = stroke, x = 'age', y = 'avg_glucose_level', color = "#9d02d7",
    ax = ax[0], edgecolor = 'black', linewidth = 1.2, alpha = 0.6
)

# Age vs BMI
sns.scatterplot(
    data = no_stroke, x = 'age', y = 'bmi', color = '#f1d295',
    alpha = 0.4, ax = ax[1]
)
sns.scatterplot(
    data = stroke, x = 'age', y = 'bmi', color = "#9d02d7",
    ax = ax[1], edgecolor = 'black', linewidth = 1.2, alpha = 0.6
)

# Glucose Levels vs BMI
sns.scatterplot(
    data = no_stroke, x = 'avg_glucose_level', y = 'bmi', color = '#f1d295',
    alpha = 0.4, ax = ax[2]
)
sns.scatterplot(
    data = stroke, x = 'avg_glucose_level', y = 'bmi', color = "#9d02d7",
    ax = ax[2], edgecolor = 'black', linewidth = 1.2, alpha = 0.6
)
    
sns.despine()

for i in range(0, 3, 1):
    ax[i].legend(['No Stroke', 'Stroke'])

fig.tight_layout()

#### 2.3 Categorical Variables

<a id="categorical"></a>

<p style="text-align:center;">Let's first investigate the target variable.</p>

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
fig.patch.set_facecolor('#faf9f7')
ax.set_facecolor('#faf9f7')

labels = ['Stroke', 'No Stroke']
colors = ["#f1d295", "#ea5f94"]
sizes = df['stroke'].value_counts()

plt.pie(sizes, explode = [0, 0.15], labels = labels, colors = colors,
           autopct = '%1.1f%%', shadow = True, startangle = 130,
           wedgeprops = {'ec': 'black'}, textprops = {'fontweight': 'medium'}
)
plt.axis('equal')
plt.title('Percentage of Strokes')

<p style="text-align:center;"><b>Gender</b></p>

In [ ]:
male_str = 0
fem_str = 0
male_nstr = 0
fem_nstr = 0

for index, row in df.iterrows():
    if row['gender'] == 'Male':
        if row['stroke'] == 1:
            male_str += 1
        else:
            male_nstr += 1
    else:
        if row['stroke'] == 1:
            fem_str += 1
        else:
            fem_nstr += 1

print(male_str, fem_str, male_nstr, fem_nstr)

In [ ]:
plt.subplots(figsize=(8,6))

stroke_matrix = np.array([[108, 2007], [141, 2854]])
labels = np.array([['Male - Stroke', 'Male - No Stroke'], ['Female - Stroke', 'Female - No Stroke']])
formatted = (np.asarray(["{0}\n{1:.0f}".format(text, data) for text, data in zip(labels.flatten(), stroke_matrix.flatten())])).reshape(2,2)


sns.heatmap(
    stroke_matrix,
    annot = formatted,
    fmt = '',
    cmap = palette,
    xticklabels = False,
    yticklabels = False,
    linecolor = 'black',
    linewidth = 1,
    annot_kws = {'fontweight': 'semibold'}
)
plt.title('Two-Way Contingency Table of Strokes by Gender', pad = 15, fontsize = 14)
plt.ylabel('Gender', fontsize = 12, labelpad = 10)
plt.xlabel('Stroke', fontsize = 12, labelpad = 10)

<p style="text-align:center;"><b>Heart Disease</b></p>

In [ ]:
heart_cont = pd.crosstab(df['heart_disease'], df['stroke'])
heart_cont

In [ ]:
plt.subplots(figsize=(8,6))

heart_matrix = np.array([[4632, 202], [229, 47]])
labels = np.array([['No Heart Disease - No Stroke', 'No Heart Disease - Stroke'], ['Heart Disease - No Stroke', 'Heart Disease - Stroke']])
formatted = (np.asarray(["{0}\n{1:.0f}".format(text, data) for text, data in zip(labels.flatten(), heart_matrix.flatten())])).reshape(2,2)

sns.heatmap(
    heart_cont,
    annot = formatted,
    fmt = '',
    cmap = palette,
    linewidth = 1,
    linecolor = 'black',
    xticklabels = False,
    yticklabels = False,
    annot_kws = {'fontweight': 'semibold'}
)
plt.ylabel('Heart Disease', labelpad = 10, fontsize = 12)
plt.xlabel('Stroke', labelpad = 10, fontsize = 12)

In [ ]:
stat, p, dof, expected = chi2_contingency(heart_cont)
stat, p

<p style="text-align:center;"><b>Hypertension</b></p>

In [ ]:
hyper_cont = pd.crosstab(df['hypertension'], df['stroke'])
hyper_cont

In [ ]:
plt.subplots(figsize=(8,6))

hyper_matrix = np.array([[4429, 183], [432, 66]])
labels = np.array([['No Hypertension - No Stroke', 'No Hypertension - Stroke'], ['Hypertension - No Stroke', 'Hypertension - Stroke']])
formatted = (np.asarray(["{0}\n{1:.0f}".format(text, data) for text, data in zip(labels.flatten(), hyper_matrix.flatten())])).reshape(2,2)

sns.heatmap(
    hyper_cont,
    annot = formatted,
    fmt = '',
    cmap = palette,
    linewidth = 1,
    linecolor = 'black',
    xticklabels = False,
    yticklabels = False,
    annot_kws = {'fontweight': 'semibold'}
)
plt.ylabel('Hypertension', labelpad = 10, fontsize = 12)
plt.xlabel('Stroke', labelpad = 10, fontsize = 12)

<p style="text-align:center;"><b>Residence Type</b></p>

In [ ]:
df.groupby('Residence_type')['stroke'].value_counts()

In [ ]:
res_cont = pd.crosstab(df['Residence_type'], df['stroke'])
res_cont

In [ ]:
plt.subplots(figsize=(8,6))

res_matrix = np.array([[2400, 114], [2461, 135]])
labels = np.array([['Rural - No Stroke', 'Rural - Stroke'], ['Urban - No Stroke', 'Urban - Stroke']])
formatted = (np.asarray(["{0}\n{1:.0f}".format(text, data) for text, data in zip(labels.flatten(), res_matrix.flatten())])).reshape(2,2)

sns.heatmap(
    res_cont,
    annot = formatted,
    fmt = '',
    cmap = palette,
    linewidth = 1,
    linecolor = 'black',
    xticklabels = False,
    yticklabels = False,
    annot_kws = {'fontweight': 'semibold'}
)
plt.ylabel('Residence Type', labelpad = 10, fontsize = 12)
plt.xlabel('Stroke', labelpad = 10, fontsize = 12)

<p style="text-align:center;"><b>Ever Married</b></p>

In [ ]:
mar_cont = pd.crosstab(df['ever_married'], df['stroke'])
mar_cont

In [ ]:
plt.subplots(figsize=(8,6))

mar_matrix = np.array([[1728, 29], [3133, 220]])
labels = np.array([['Never Married - No Stroke', 'Never Married - Stroke'], ['Married - No Stroke', 'Married - Stroke']])
formatted = (np.asarray(["{0}\n{1:.0f}".format(text, data) for text, data in zip(labels.flatten(), mar_matrix.flatten())])).reshape(2,2)

sns.heatmap(
    mar_cont,
    annot = formatted,
    fmt = '',
    cmap = palette,
    linewidth = 1,
    linecolor = 'black',
    xticklabels = False,
    yticklabels = False,
    annot_kws = {'fontweight': 'semibold'}
)
plt.ylabel('Ever Married', labelpad = 10, fontsize = 12)
plt.xlabel('Stroke', labelpad = 10, fontsize = 12)

<p style="text-align:center;"><b>Smoking Status</b></p>

In [ ]:
df['smoking_status'].unique()

In [ ]:
df.groupby('smoking_status')['stroke'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
fig.patch.set_facecolor('#faf9f7')
ax.set_facecolor('#faf9f7')

bar_pal = ["#c8c14f", "#fa8775"]

s = sns.countplot(
    data = df, x = 'smoking_status', hue = 'stroke', palette = bar_pal,
    linewidth = 1.2, ec = 'black'
)

for i in ['top', 'right', 'bottom', 'left']:
    ax.spines[i].set_visible(False)

plt.legend(['No Stroke', 'Stroke'])
plt.title("Smoking Status' Effect on Stroke", size = 16, weight = 'bold', pad = 12)
plt.xlabel('Smoking Status', size = 12, labelpad = 12)
plt.ylabel('Count', size = 12, labelpad = 12)

for i in s.patches:
    s.annotate(format(i.get_height(), '.0f'),  (i.get_x() + i.get_width() / 2., i.get_height()), ha = 'center', va = 'center', xytext = (0, 9), textcoords = 'offset points')

fig.tight_layout()

<p style="text-align:center;"><b>Work Type</b></p>

In [ ]:
df['work_type'].unique()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
fig.patch.set_facecolor('#faf9f7')
ax.set_facecolor('#faf9f7')

bar_pal = ["#c8c14f", "#fa8775"]

w = sns.countplot(
    data = df, x = 'work_type', hue = 'stroke', palette = bar_pal,
    linewidth = 1.2, ec = 'black'
)

for i in ['top', 'right', 'bottom', 'left']:
    ax.spines[i].set_visible(False)

plt.legend(['No Stroke', 'Stroke'])
plt.title("Work Type's Effect on Stroke", size = 16, weight = 'bold', pad = 12)
plt.xlabel('Work Type', size = 12, labelpad = 12)
plt.ylabel('Count', size = 12, labelpad = 12)

for i in w.patches:
    w.annotate(format(i.get_height(), '.0f'),  (i.get_x() + i.get_width() / 2., i.get_height()), ha = 'center', va = 'center', xytext = (0, 9), textcoords = 'offset points')

fig.tight_layout()

# **3. Bonus EDA - Odds**

<a id="bonus"></a>

In [ ]:
gen_odds = (108 * 2854) / (141 * 2007)

heart_odds = (229 * 202) / (4632 * 47)

hyper_odds = (432 * 183) / (4429 * 66)

res_odds = (2400 * 135) / (2461 * 114)

mar_odds = (1728 * 220) / (3133 * 29)

d = {
    'Features': ['Gender', 'Heart Disease', 'Hypertension',
                'Residence', 'Married'],
    'Odds': [gen_odds, heart_odds, hyper_odds, res_odds, mar_odds]
}

odds_df = pd.DataFrame(data = d)
odds_df

<p style="text-align:center;">Most features seem to have little difference in their odds. The ever married variable has a 4 to 1 odds of having a stroke for individuals that were never married.</p>

# **4. Feature Engineering**

<a id="features"></a>

In [ ]:
df = pd.get_dummies(df, columns = ['gender', 'work_type', 'Residence_type', 'smoking_status'], prefix = ['sex', 'work', 'residence', 'smoke'])
df.head()

In [ ]:
df['ever_married'] = df['ever_married'].apply(lambda x: 1 if x == 'Yes' else 0)
df.head()

<p style="text-align:center;">Scale continuous features if you are using distance-based algorithms such as k-nearest neighbors. Since we will be using knn, we will scale our features.</p>

In [ ]:
num_cols = ['age', 'avg_glucose_level', 'bmi']

scaler = StandardScaler()

df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
df.head()

In [ ]:
df = df.drop('id', axis = 1)
df.head()

# **5. Model Building**

<a id="models"></a>

<p style="text-align:center;">Random state can be any number</p>

<p style="text-align:center;">SMOTE helps with the severe imabalance of target variable. If we remember, only 5% of all cases actually included a stroke. It can help improve recall; that is- predict the number of people who actually had a stroke. Since we would care more about predicting who might have a stroke rather than who might not have one, SMOTE can help us accomplish that. We could try two different models using the original data and the oversampled data to determine if it is effective.</p>

In [ ]:
x = df.drop('stroke', axis = 1)
y = df['stroke']


smote = SMOTE()

x_oversample, y_oversample = smote.fit_resample(x, y)

print(y.value_counts())
print(y_oversample.value_counts())

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_oversample, y_oversample, test_size = 0.2, random_state = 0)

Metrics:

* Precision is the total number of people the model correctly identified as having a stroke out of all the people PREDICTED to have a stroke

* Recall is the total number of people the model correctly identified as a having a stroke out of all the people who ACTUALLY HAD a stroke.

* Accuracy is the total number of correct predictions divided by the total number of predictions.

* It is not possible to achieve both a high precision and a high recall value- we must determine which is more important for us in our model.

* F1 gives us the harmonic mean of precision and recall (Aim for a high F1 value to indicate a good precision and a good recall value).

* ROC (Receiver Operating Characteristic) Curve is a plot betwen the True Positive Rate on the y-axis and the False Positive Rate on the x-axis. A plot with the graph closer to the left and top axes is indicative of a better model.

* AUC (Area Under Curve) values range from 0 to 1 with higher scores indicating a better model. The diagonal line on ROC curves usually represents a random model with an AUC of 0.5. (Would definitely want our model's AUC to be higher than 0.5, since that would signify it is better than random chance.

* PRC (Precision-Recall Curves) plot values of precision scores on the y-axis and recall on the x-axis. A plot with the graph closer to the top and right axes is indicative of a better model. As with ROC curves, we should aim for a high AUC.

#### 5.1 Logistic Regression

<a id="logreg"></a>

In [ ]:
log = LogisticRegression()
log.fit(x_train, y_train)
y_pred_log = log.predict(x_test)
cr = classification_report(y_test, y_pred_log)
print(cr)

In [ ]:
print('Precision Score: ', round(precision_score(y_test, y_pred_log), 2))
print('Recall Score: ', round(recall_score(y_test, y_pred_log), 2))
print('F1 Score: ', round(f1_score(y_test, y_pred_log), 2))
print('Accuracy Score: ', round(accuracy_score(y_test, y_pred_log), 2))
print('ROC AUC: ', round(roc_auc_score(y_test, y_pred_log), 2))

In [ ]:
plot_roc_curve(log, x_test, y_test)

In [ ]:
sns.heatmap(
    confusion_matrix(y_test, y_pred_log),
    cmap = palette,
    annot = True,
    fmt = 'd',
    yticklabels = ['No Stroke', 'Stroke'],
    xticklabels = ['Pred No Stroke', 'Pred Stroke']
)

#### 5.2 Random Forest

<a id="forest"></a>

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)
cr_rf = classification_report(y_test, y_pred_rf)
print(cr_rf)

In [ ]:
print('Precision Score: ', round(precision_score(y_test, y_pred_rf), 2))
print('Recall Score: ', round(recall_score(y_test, y_pred_rf), 2))
print('F1 Score: ', round(f1_score(y_test, y_pred_rf), 2))
print('Accuracy Score: ', round(accuracy_score(y_test, y_pred_rf), 2))
print('ROC AUC: ', round(roc_auc_score(y_test, y_pred_rf), 2))

In [ ]:
plot_roc_curve(rf, x_test, y_test)

In [ ]:
sns.heatmap(
    confusion_matrix(y_test, y_pred_rf),
    cmap = palette,
    annot = True,
    fmt = 'd',
    yticklabels = ['No Stroke', 'Stroke'],
    xticklabels = ['Pred No Stroke', 'Pred Stroke']
)

#### 5.3 K-Nearest Neighbors

<a id="neighbors"></a>

In [ ]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test)
cr_knn = classification_report(y_test, y_pred_knn)
print(cr_knn)

In [ ]:
print('Precision Score: ', round(precision_score(y_test, y_pred_knn), 2))
print('Recall Score: ', round(recall_score(y_test, y_pred_knn), 2))
print('F1 Score: ', round(f1_score(y_test, y_pred_knn), 2))
print('Accuracy Score: ', round(accuracy_score(y_test, y_pred_knn), 2))
print('ROC AUC: ', round(roc_auc_score(y_test, y_pred_knn), 2))

In [ ]:
plot_roc_curve(knn, x_test, y_test)

In [ ]:
sns.heatmap(
    confusion_matrix(y_test, y_pred_knn),
    cmap = palette,
    annot = True,
    fmt = 'd',
    yticklabels = ['No Stroke', 'Stroke'],
    xticklabels = ['Pred No Stroke', 'Pred Stroke']
)

#### 5.4 AdaBoost

<a id="ada"></a>

In [ ]:
ada = AdaBoostClassifier()
ada.fit(x_train, y_train)
y_pred_ada = ada.predict(x_test)
cr_ada = classification_report(y_test, y_pred_ada)
print(cr_ada)

In [ ]:
print('Precision Score: ', round(precision_score(y_test, y_pred_ada), 2))
print('Recall Score: ', round(recall_score(y_test, y_pred_ada), 2))
print('F1 Score: ', round(f1_score(y_test, y_pred_ada), 2))
print('Accuracy Score: ', round(accuracy_score(y_test, y_pred_ada), 2))
print('ROC AUC: ', round(roc_auc_score(y_test, y_pred_ada), 2))

In [ ]:
plot_roc_curve(ada, x_test, y_test)

In [ ]:
sns.heatmap(
    confusion_matrix(y_test, y_pred_ada),
    cmap = palette,
    annot = True,
    fmt = 'd',
    yticklabels = ['No Stroke', 'Stroke'],
    xticklabels = ['Pred No Stroke', 'Pred Stroke']
)

# **6. Conclusion**

<a id="conclusion"></a>

<p style="text-align:center;">Given its high scores across the board, particularly in recall, I would say we should choose the random forest classifier. With such a high F1 score we can be quite confident in our pick. This model should be quite reliable at predicting which people are most at risk for having a stroke and which do not need unnecessary treatment.</p>